In [266]:
import polars as pl
import pathlib
import sys
import json
import statsmodels.api as sm
import statsmodels.formula.api as smf
import numpy as np
import pandas as pd
import xgboost as xgb
import plotly.express as px

In [267]:
sys.path.append('/Users/urolyi1/repos/march-madness')

In [268]:
import kaggle_2025.preprocess_data as preprocess_data
import uro_cbb.model_utils as model_utils

In [269]:
%load_ext autoreload
%reload_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [270]:
preprocessed_dfs = {}
for year in range(2015, 2025):
    if year == 2020:
        continue
    preprocessed_dfs[year] = preprocess_data.preprocess_data(year)
    

ERROR:root:Missing names: {'ut rio grande valley', 'mississippi valley st.', 'arkansas pine bluff', 'southeast missouri st.', 'st. francis ny', 'bethune cookman', 'illinois chicago', 'tennessee martin', 'louisiana monroe', 'saint francis', 'cal st. bakersfield'}
ERROR:root:Missing names: {'mississippi valley st.', 'arkansas pine bluff', 'louisiana lafayette', 'southeast missouri st.', 'st. francis ny', 'st. francis pa', 'bethune cookman', 'illinois chicago', 'tennessee martin', 'arkansas little rock', 'texas pan american', 'louisiana monroe', 'cal st. bakersfield'}
ERROR:root:Missing names: {'ut rio grande valley', 'mississippi valley st.', 'arkansas pine bluff', 'southeast missouri st.', 'st. francis ny', 'bethune cookman', 'illinois chicago', 'tennessee martin', 'louisiana monroe', 'saint francis', 'cal st. bakersfield'}
ERROR:root:Missing names: {'ut rio grande valley', 'mississippi valley st.', 'arkansas pine bluff', 'louisiana lafayette', 'southeast missouri st.', 'st. francis ny'

In [271]:
womens_preprocessed_dfs = {}
for year in range(2016, 2025):
    if year == 2020:
        continue
    womens_preprocessed_dfs[year] = preprocess_data.preprocess_womens_data(year)
    

ERROR:root:Missing names: {'ut rio grande valley', 'mississippi valley st.', 'arkansas pine bluff', 'southeast missouri st.', 'tarleton st.', 'queens', 'illinois chicago', 'texas a&m commerce', 'tennessee martin', 'bethune cookman', 'louisiana monroe', 'saint francis', 'cal st. bakersfield'}
ERROR:root:Missing names: {'ut rio grande valley', 'mississippi valley st.', 'arkansas pine bluff', 'southeast missouri st.', 'tarleton st.', 'queens', 'illinois chicago', 'texas a&m commerce', 'tennessee martin', 'bethune cookman', 'louisiana monroe', 'saint francis', 'cal st. bakersfield'}
ERROR:root:Missing names: {'ut rio grande valley', 'mississippi valley st.', 'arkansas pine bluff', 'southeast missouri st.', 'tarleton st.', 'queens', 'illinois chicago', 'texas a&m commerce', 'tennessee martin', 'bethune cookman', 'louisiana monroe', 'saint francis', 'cal st. bakersfield'}
ERROR:root:Missing names: {'ut rio grande valley', 'mississippi valley st.', 'arkansas pine bluff', 'southeast missouri s

In [272]:
training_df = pl.concat([womens_preprocessed_dfs[year] for year in range(2016, 2025) if year != 2020], how='diagonal')
training_df = training_df.to_pandas()
x_cols = [column for column in training_df.columns[8:32] if '_2' not in column]
x_2_cols = [column for column in training_df.columns if '_2' in column]
assert len(x_cols) == len(x_2_cols)
dup_training_df = training_df.copy()
dup_training_df[x_cols] = training_df[x_2_cols]
dup_training_df[x_2_cols] = training_df[x_cols]
dup_training_df['Result'] = 1 - training_df['Result']
final_training_df = pd.concat([training_df, dup_training_df]).reset_index(drop=True)
# test_df = womens_preprocessed_dfs[2024]
womens_model = smf.glm(
    "Result ~ adjoe + adjoe_2 + adjde + adjde_2 ",
    data=final_training_df,
    family=sm.families.Binomial(link=sm.families.links.Logit())
)
results = womens_model.fit()

# print(model_utils.evaluate_model(
#     test_df.with_columns(
#         pl.Series(values=results.predict(test_df.to_pandas()).values, name='pred'),
#     ),
#     cap=.95
# ))
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                 Result   No. Observations:                 1000
Model:                            GLM   Df Residuals:                      995
Model Family:                Binomial   Df Model:                            4
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -505.32
Date:                Thu, 20 Mar 2025   Deviance:                       1010.6
Time:                        13:33:47   Pearson chi2:                 1.01e+03
No. Iterations:                     5   Pseudo R-squ. (CS):             0.3132
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   8.882e-15      2.637   3.37e-15      1.000      -5.168       5.168
adjoe          0.0707      0.011      6.641      0.000       0.050       0.092
adjoe_2       -0.0707      0.011     -6.641      0.000      -0.092      -0.050
adjde         -0.0740      0.014     -5.182      0.000      -0.102      -0.046
adjde_2        0.0740      0.014      5.182      0.000       0.046       0.102
==============================================================================
"""

In [273]:
# results.save('../kaggle_2025/models/womens_barttorvik.pkl')

In [274]:
training_df = pl.concat([preprocessed_dfs[year] for year in range(2016, 2025) if year != 2020], how='diagonal')
training_df = training_df.to_pandas()
x_cols = [column for column in training_df.columns[8:] if '_2' not in column]
x_2_cols = [column for column in training_df.columns if '_2' in column]
assert len(x_cols) == len(x_2_cols)
dup_training_df = training_df.copy()
dup_training_df[x_cols] = training_df[x_2_cols]
dup_training_df[x_2_cols] = training_df[x_cols]
dup_training_df['Result'] = 1 - training_df['Result']
final_training_df = pd.concat([training_df, dup_training_df]).reset_index(drop=True)
# test_df = preprocessed_dfs[2025]
mens_model = smf.glm(
    "Result ~ adjoe + adjoe_2 + adjde + adjde_2 + AdjO + AdjO_2 + AdjD + AdjD_2",
    data=final_training_df,
    family=sm.families.Binomial(link=sm.families.links.Logit())
)
results = mens_model.fit()

# print(model_utils.evaluate_model(
#     test_df.with_columns(
#         pl.Series(values=results.predict(test_df.to_pandas()).values, name='pred'),
#     ),
#     cap=.95
# ))
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                 Result   No. Observations:                  874
Model:                            GLM   Df Residuals:                      865
Model Family:                Binomial   Df Model:                            8
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -488.58
Date:                Thu, 20 Mar 2025   Deviance:                       977.16
Time:                        13:33:47   Pearson chi2:                     896.
No. Iterations:                     5   Pseudo R-squ. (CS):             0.2353
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept  -5.107e-15      3.293  -1.55e-15      1.000      -6.454       6.454
adjoe          0.1508      0.086      1.749      0.080      -0.018       0.320
adjoe_2       -0.1508      0.086     -1.749      0.080      -0.320       0.018
adjde         -0.1221      0.099     -1.237      0.216      -0.316       0.071
adjde_2        0.1221      0.099      1.237      0.216      -0.071       0.316
AdjO          -0.0519      0.091     -0.571      0.568      -0.230       0.126
AdjO_2         0.0519      0.091      0.571      0.568      -0.126       0.230
AdjD           0.0405      0.092      0.441      0.659      -0.140       0.221
AdjD_2        -0.0405      0.092     -0.441      0.659      -0.221       0.140
==============================================================================
"""

In [275]:
# results.save('../kaggle_2025/models/mens_barttorvik_kenpom.pkl')

In [276]:
def _merge_with_kaggle_names(
    df: pl.DataFrame, kaggle_names_df: pl.DataFrame, team_col: str
) -> pl.DataFrame:
    kaggle_names_df = kaggle_names_df.with_columns(
        pl.col("Team").alias("__KAGGLE_TEAM_SPELLING_VARIATION"),
        pl.col("TeamID").alias("__KAGGLE_TEAM_ID"),
        pl.col("TeamName").alias("__KAGGLE_TEAM_NAME"),
    ).select(pl.exclude("Team", "TeamID", "TeamName"))
    processed_df = (
        df.with_columns(
            pl.col(team_col).str.to_lowercase().alias(f"{team_col}_lower"),
        )
        .join(
            kaggle_names_df,
            left_on=f"{team_col}_lower",
            right_on="__KAGGLE_TEAM_SPELLING_VARIATION",
            how="inner",
        )
        .with_columns(
            pl.col("__KAGGLE_TEAM_NAME").alias(team_col),
            pl.col("__KAGGLE_TEAM_ID").alias(f"{team_col}_id"),
        )
        .select(
            pl.exclude(
                [
                    f"{team_col}_lower",
                    "__KAGGLE_TEAM_SPELLING_VARIATION",
                    "__KAGGLE_TEAM_ID",
                    "__KAGGLE_TEAM_NAME",
                ]
            )
        )
    )
    if processed_df.shape[0] != df.shape[0]:
        df_names = set(df.select(pl.col(team_col)).to_series())
        kaggle_names = set(
            kaggle_names_df.select(
                pl.col("__KAGGLE_TEAM_SPELLING_VARIATION")
            ).to_series()
        )
        missing_names = df_names - kaggle_names
        print(missing_names)
    return processed_df

In [277]:
module_dir = pathlib.Path('.').absolute()
kaggle_names_df = pl.read_csv(
    module_dir.parent / "data/kaggle_2025/raw/MTeamSpellings.csv",
    schema={"Team": pl.Utf8, "TeamID": pl.Int16},
).join(
    pl.read_csv(module_dir.parent / "data/kaggle_2025/raw/MTeams.csv").select(
        pl.col("TeamID"), pl.col("TeamName")
    ),
    on="TeamID",
)
kaggle_names_df

Team,TeamID,TeamName
str,i16,str
"""a&m-corpus chris""",1394,"""TAM C. Christi"""
"""a&m-corpus christi""",1394,"""TAM C. Christi"""
"""abilene chr""",1101,"""Abilene Chr"""
"""abilene christian""",1101,"""Abilene Chr"""
"""abilene-christian""",1101,"""Abilene Chr"""
…,…,…
"""youngstown st""",1464,"""Youngstown St"""
"""youngstown st.""",1464,"""Youngstown St"""
"""youngstown state""",1464,"""Youngstown St"""


In [278]:
kenpom_df =(
    pl.read_parquet('../data/kenpom/raw/kenpom_2025.parquet')
    .with_columns(
        pl.col("Team").str.to_lowercase().alias("Team")
    )
    .pipe(_merge_with_kaggle_names, kaggle_names_df, "Team")
    .select(
        pl.col('Team'),
        pl.col('Team_id').alias('TeamID'),
        pl.col('AdjO'),
        pl.col('AdjD')
    )
)
kenpom_df

{'ut rio grande valley', 'mississippi valley st.', 'southeast missouri', 'arkansas pine bluff', 'tarleton st.', 'queens', 'illinois chicago', 'bethune cookman', 'tennessee martin', 'louisiana monroe', 'saint francis', 'cal st. bakersfield'}


Team,TeamID,AdjO,AdjD
str,i16,f64,f64
"""Abilene Chr""",1101,97.6,102.1
"""Air Force""",1102,98.9,111.5
"""Akron""",1103,113.7,107.1
"""Alabama""",1104,126.5,96.6
"""Alabama A&M""",1105,92.0,115.5
…,…,…,…
"""Wright St""",1460,109.5,114.1
"""Wyoming""",1461,103.6,105.5
"""Xavier""",1462,115.5,98.3


In [279]:
barttorvik_df = (
    pl.read_parquet('../data/barttorvik/raw/mens/barttorvik_2025.parquet')
    .with_columns(
        pl.col("team").str.to_lowercase().alias("team")
    )
    .pipe(_merge_with_kaggle_names, kaggle_names_df, "team")
    .select(
        pl.col('team').alias('Team'),
        pl.col('team_id').alias('TeamID'),
        pl.col('adjoe'),
        pl.col('adjde')
    )
)
barttorvik_df

{'ut rio grande valley', 'mississippi valley st.', 'arkansas pine bluff', 'southeast missouri st.', 'tarleton st.', 'queens', 'illinois chicago', 'texas a&m commerce', 'tennessee martin', 'bethune cookman', 'louisiana monroe', 'saint francis', 'cal st. bakersfield'}


Team,TeamID,adjoe,adjde
str,i16,f32,f32
"""Abilene Chr""",1101,98.199997,101.699997
"""Air Force""",1102,98.800003,109.599998
"""Akron""",1103,113.300003,106.300003
"""Alabama""",1104,127.0,96.199997
"""Alabama A&M""",1105,92.800003,114.699997
…,…,…,…
"""Wright St""",1460,108.699997,112.5
"""Wyoming""",1461,103.199997,105.099998
"""Xavier""",1462,116.0,98.699997


In [280]:
cleaned_df = kenpom_df.join(
    barttorvik_df,
    on=['Team', 'TeamID'],
    how='inner'
)
# cleaned_df =barttorvik_df
cleaned_df

Team,TeamID,AdjO,AdjD,adjoe,adjde
str,i16,f64,f64,f32,f32
"""Abilene Chr""",1101,97.6,102.1,98.199997,101.699997
"""Air Force""",1102,98.9,111.5,98.800003,109.599998
"""Akron""",1103,113.7,107.1,113.300003,106.300003
"""Alabama""",1104,126.5,96.6,127.0,96.199997
"""Alabama A&M""",1105,92.0,115.5,92.800003,114.699997
…,…,…,…,…,…
"""Wright St""",1460,109.5,114.1,108.699997,112.5
"""Wyoming""",1461,103.6,105.5,103.199997,105.099998
"""Xavier""",1462,115.5,98.3,116.0,98.699997


In [281]:
# cleaned_df.write_csv('../data/kaggle_2025/cleaned/mens/barttorvik_kenpom.csv')
# cleaned_df.write_parquet('../data/kaggle_2025/cleaned/mens/barttorvik_kenpom.parquet')

In [282]:
module_dir = pathlib.Path('.').absolute()
kaggle_womens_names_df = pl.read_csv(
    module_dir.parent / "data/kaggle_2025/raw/WTeamSpellings.csv",
    schema={"Team": pl.Utf8, "TeamID": pl.Int16},
).join(
    pl.read_csv(module_dir.parent / "data/kaggle_2025/raw/WTeams.csv").select(
        pl.col("TeamID"), pl.col("TeamName")
    ),
    on="TeamID",
)
kaggle_womens_names_df

Team,TeamID,TeamName
str,i16,str
"""a&m-corpus chris""",3394,"""TAM C. Christi"""
"""a&m-corpus christi""",3394,"""TAM C. Christi"""
"""abilene chr""",3101,"""Abilene Chr"""
"""abilene christian""",3101,"""Abilene Chr"""
"""abilene-christian""",3101,"""Abilene Chr"""
…,…,…
"""youngstown st""",3464,"""Youngstown St"""
"""youngstown st.""",3464,"""Youngstown St"""
"""youngstown state""",3464,"""Youngstown St"""


In [283]:
barttorvik_df = (
    pl.read_parquet('../data/barttorvik/raw/womens/barttorvik_2025.parquet')
    .with_columns(
        pl.col("team").str.to_lowercase().alias("team")
    )
    .pipe(_merge_with_kaggle_names, kaggle_womens_names_df, "team")
    .select(
        pl.col('team').alias('Team'),
        pl.col('team_id').alias('TeamID'),
        pl.col('adjoe'),
        pl.col('adjde')
    )
)
barttorvik_df

{'ut rio grande valley', 'mississippi valley st.', 'arkansas pine bluff', 'southeast missouri st.', 'tarleton st.', 'queens', 'illinois chicago', 'texas a&m commerce', 'tennessee martin', 'bethune cookman', 'louisiana monroe', 'saint francis', 'cal st. bakersfield'}


Team,TeamID,adjoe,adjde
str,i16,f32,f32
"""Abilene Chr""",3101,93.199997,94.599998
"""Air Force""",3102,93.199997,92.699997
"""Akron""",3103,87.699997,101.800003
"""Alabama""",3104,113.800003,79.599998
"""Alabama A&M""",3105,86.699997,95.400002
…,…,…,…
"""Wright St""",3460,86.0,103.0
"""Wyoming""",3461,99.900002,86.0
"""Xavier""",3462,81.900002,87.900002


In [284]:
cleaned_df = barttorvik_df
# cleaned_df.write_csv('../data/kaggle_2025/cleaned/womens/barttorvik.csv')
# cleaned_df.write_parquet('../data/kaggle_2025/cleaned/womens/barttorvik.parquet')